# Write Julia-C++ wrapper for ghostbasil

`ml cmake/3.11.1 gcc/7.1.0 julia/1.8.4 eigen/3.4.0`

1. We will be using [CxxWrap.jl](https://github.com/JuliaInterop/CxxWrap.jl) to call C++ code within Julia. 
2. We need a working `libcxxwrap_julia_jll.jl`. I simply installed the jll package
```
add https://github.com/barche/libcxxwrap_julia_jll.jl.git
```
3. Once it's done, we need CMake to discover `libcxxwrap-julia`. This can be done by adding `-DCMAKE_PREFIX_PATH=/path/to/libcxxwrap-julia-prefix` to the `cmake` command where `/path/to/libcxxwrap-julia-prefix` is obtained from
```
julia> using CxxWrap
julia> CxxWrap.prefix_path()
```

## Hello world example

Here is my `hello.cpp` file:
```cpp
#include <string>
#include "jlcxx/jlcxx.hpp"

std::string greet()
{
   return "hello, world";
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
  mod.method("greet", &greet);
}
```

And here is the `CMakeLists.txt` file:

```cmake
project(HelloWorld)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(hello SHARED hello.cpp)

target_link_libraries(hello JlCxx::cxxwrap_julia)

install(TARGETS
  hello
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```

Given these 2 files, we build the shared library (`.io`) file as
```
mkdir build && cd build
cmake \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/artifacts/d25a86126e4d79572040fbf3f1fcda158a703a4a \
    ../
cmake --build . --config Release
```
Finally, we can call hello world in Julia:

In [1]:
# Load the module and generate the functions
module CppHello
    using CxxWrap
    @wrapmodule(() -> "/home/users/bbchu/ghostbasilwrap/build/lib/libhello")

    function __init__()
        @initcxx
    end
end

# Call greet and show the result
CppHello.greet()

"hello, world"

## Pass matrix from Julia into C++, calculate the sum, and return answer

First create `sumarray.cpp`

```cpp
#include "jlcxx/jlcxx.hpp"
#include <vector>
#include <iostream>
using namespace std;

double cpp_mat_sum(jlcxx::ArrayRef<double, 2> x) {
    cout << &x[0] << endl; // print memory address of x
    cout << x.size() << endl; // print length of x
    double total = 0;
    for(auto xij : x) {
        total += xij;
    }
    return total;
}

std::vector<double> get_custom_return_vec(jlcxx::ArrayRef<double, 2> x) {
    std::vector<double> result;
    result.push_back(cpp_mat_sum(x));
    result.push_back(x[0]);
    result.push_back(x[1]);
    return result;
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.method("cpp_mat_sum", &cpp_mat_sum);
    mod.method("get_custom_return_vec", &get_custom_return_vec);
}
```

Then create `CMakeLists.txt` file

```cmake
project(Examples)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(hello SHARED hello.cpp)
add_library(sumarray SHARED sumarray.cpp)

target_link_libraries(hello JlCxx::cxxwrap_julia)
target_link_libraries(sumarray JlCxx::cxxwrap_julia)

install(TARGETS
  hello sumarray 
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```
Given these 2 files, we build the shared library (`.io`) file as
```
mkdir build && cd build
cmake \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/artifacts/d25a86126e4d79572040fbf3f1fcda158a703a4a \
    ../
cmake --build . --config Release
```
Finally, we can call both `hello` and `sumarray`:

In [1]:
module CppHello
    using CxxWrap
    @wrapmodule(() -> "/home/users/bbchu/ghostbasilwrap/build/lib/libhello")

    function __init__()
        @initcxx
    end
end

# Call greet and show the result
CppHello.greet()

"hello, world"

In [2]:
module CppSumArray
    using CxxWrap
    @wrapmodule(() -> "/home/users/bbchu/ghostbasilwrap/build/lib/libsumarray")

    function __init__()
        @initcxx
    end
end

x = randn(10000, 10000)
CppSumArray.cpp_mat_sum(x)

0x7fbaf050b040
100000000


-9379.300114470054

In [3]:
sum(x)

-9379.300114469586

In [4]:
pointer(x)

Ptr{Float64} @0x00007fbaf050b040

In [5]:
CppSumArray.get_custom_return_vec(x)

0x7fbaf050b040
100000000


3-element CxxWrap.StdLib.StdVectorAllocated{Float64}:
 -9379.300114470054
    -0.4856036463361442
     0.6013781816541801

## Pass matrix from Julia into C++ as a Eigen matrix

+ [this tutorial](https://www.youtube.com/watch?v=VoXmXtqLhdo&ab_channel=TheJuliaProgrammingLanguage) which wraps the Eigen library (starging at 1:13:00) was especially helpful.


1. First create `eigen_matrix.cpp`

```cpp
#include <jlcxx/jlcxx.hpp>
#include <jlcxx/stl.hpp>
#include <Eigen/Dense>

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
  mod.add_type<Eigen::MatrixXd>("MatrixXd", jlcxx::julia_type("AbstractMatrixXd"))
    .constructor<int_t, int_t>()
    .method("cols", &Eigen::MatrixXd::cols)
    .method("rows", &Eigen::MatrixXd::rows)
    .method("norm", &Eigen::MatrixXd::norm)
    .method("setConstant", static_cast<Eigen::MatrixXd& (Eigen::MatrixXd::*)(const double&)>(&Eigen::MatrixXd::setConstant))
  ;

  mod.set_override_module(jl_base_module);
  mod.method("getindex", [](const Eigen::MatrixXd& m, int_t i, int_t j) { return m(i-1, j-1); });
  mod.method("setindex!", [](Eigen::MatrixXd& m, double value, int_t i, int_t j) { m(i-1, j-1) = value; });
  mod.unset_override_module();
}
```

2. Then create `CMakeLists.txt` file

```cmake
project(JlEigen)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(Eigen3 3.3 REQUIRED NO_MODULE)
find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(eigen_matrix SHARED eigen_matrix.cpp)
target_link_libraries(eigen_matrix JlCxx::cxxwrap_julia Eigen3::Eigen)

install(TARGETS
  eigen_matrix 
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```
3. Build the shared library (`.io`) file as
```
mkdir build && cd build
cmake \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/artifacts/d25a86126e4d79572040fbf3f1fcda158a703a4a \
    /home/users/bbchu/ghostbasilwrap \
    -DEigen3_DIR=/share/software/user/open/eigen/3.4.0/share/eigen3/cmake \
    ../
cmake --build . --config Release
```
4. Finally, we call `eigen_matrix` in Julia as:

In [6]:
module CppEigenMatrix
    using CxxWrap
    abstract type AbstractMatrixXd <: AbstractMatrix{Float64} end

    @wrapmodule(() -> "/home/users/bbchu/ghostbasilwrap/build/lib/libeigen_matrix")

    function __init__()
        @initcxx
    end

    Base.size(m::MatrixXd) = (rows(m),cols(m))
    Base.IndexStyle(::Type{<:MatrixXd})=IndexCartesian()
end

Main.CppEigenMatrix

In [7]:
m = CppEigenMatrix.MatrixXd(2, 2)
m[1, 1] = 1
m[1, 2] = 2
m[2, 1] = 3
m[2, 2] = 4
m

2×2 Main.CppEigenMatrix.MatrixXdAllocated:
 1.0  2.0
 3.0  4.0

In [8]:
m .= rand(2, 2)
m

2×2 Main.CppEigenMatrix.MatrixXdAllocated:
 0.117769  0.400588
 0.581882  0.883709

In [9]:
m1 = CppEigenMatrix.MatrixXd(2, 2)
m2 = CppEigenMatrix.MatrixXd(3, 3)
m3 = CppEigenMatrix.MatrixXd(4, 4)
blocks = [m1, m2, m3]

3-element Vector{Main.CppEigenMatrix.MatrixXdAllocated}:
 [6.93878695892835e-310 3.8126295e-316; 4.35062666e-316 4.14520603e-316]
 [6.93878695892993e-310 1.402447575785111e-307 3.37845646e-316; 4.25517397e-316 4.2538981e-316 4.06314054e-316; 2.2089196780820652e161 1.63e-322 3.95e-322]
 [6.9387869589331e-310 0.0 1.32543386e-316 0.0; 4.12702283e-316 0.0 0.0 0.0; 4.25149615e-316 4.09589393e-316 4.246258147e-314 4.37705957e-316; 4.07452855e-316 4.07452855e-316 0.0 2.121995791e-314]

In [10]:
typeof(blocks)

Vector{MatrixXdAllocated} (alias for Array{Main.CppEigenMatrix.MatrixXdAllocated, 1})

## "Template types": pass matrix from Julia into C++ as a Eigen matrix

1. First create `eigen_matrix.cpp`

```cpp
#include <jlcxx/jlcxx.hpp>
#include <jlcxx/stl.hpp>
#include <Eigen/Dense>

namespace jleigen
{
  struct WrapMatrix
  {
    template<typename TypeWrapperT>
    void operator()(TypeWrapperT&& wrapped)
    {
      using WrappedT = typename TypeWrapperT::type; // WrappedT is basically Eigen::MatrixXd and its relatives
      using ScalarT = typename WrappedT::Scalar;
      wrapped.template constructor<Eigen::Index, Eigen::Index>();
      wrapped.method("cols", &WrappedT::cols);
      wrapped.method("rows", &WrappedT::rows);
      wrapped.method("norm", &WrappedT::norm);
      wrapped.method("setConstant", static_cast<WrappedT& (WrappedT::*)(const ScalarT&)>(&WrappedT::setConstant));
 
      wrapped.module().set_override_module(jl_base_module);
      wrapped.module().method("getindex", [](const WrappedT& m, int_t i, int_t j) { return m(i-1,j-1); });
      wrapped.module().method("setindex!", [](WrappedT& m, ScalarT value, int_t i, int_t j) { m(i-1,j-1) = value; });
      wrapped.module().unset_override_module();
 
      wrapped.module().method("toString", [] (const WrappedT& m)
      {
        std::stringstream stream;
        stream << m;
        return stream.str();
      });
    }
  };
}
 
namespace jlcxx
{
  template<typename T>
  struct BuildParameterList<Eigen::Matrix<T, Eigen::Dynamic, Eigen::Dynamic>>
  {
    typedef ParameterList<T> type;
  };
}
 
JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
  using jlcxx::Parametric;
  using jlcxx::TypeVar;
  mod.add_type<Parametric<TypeVar<1>>>("Matrix", jlcxx::julia_type("AbstractMatrix"))
    .apply<Eigen::Matrix<double, Eigen::Dynamic, Eigen::Dynamic>, Eigen::Matrix<float, Eigen::Dynamic, Eigen::Dynamic>>(jleigen::WrapMatrix());
}
```

2. Then create `CMakeLists.txt` file

```cmake
project(JlEigen)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(Eigen3 3.3 REQUIRED NO_MODULE)
find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(eigen_matrix SHARED eigen_matrix.cpp)
target_link_libraries(eigen_matrix JlCxx::cxxwrap_julia Eigen3::Eigen)

install(TARGETS
  eigen_matrix 
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```
3. Build the shared library (`.io`) file as
```
mkdir build && cd build
cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override /home/users/bbchu/ghostbasilwrap -DEigen3_DIR=/share/software/user/open/eigen/3.4.0/share/eigen3/cmake
cmake --build . --config Release
```
4. Finally, we call `eigen_matrix` in Julia as:

In [1]:
module CppEigenMatrix
    using CxxWrap

    @wrapmodule("/home/users/bbchu/ghostbasilwrap/build/lib/libeigen_matrix")

    function __init__()
        @initcxx
    end

    Base.size(m::Matrix) = (rows(m), cols(m))
    Base.IndexStyle(::Type{<:Matrix})=IndexCartesian()
end

Main.CppEigenMatrix

In [2]:
m = CppEigenMatrix.Matrix{Float64}(2, 2)
m

2×2 Main.CppEigenMatrix.MatrixAllocated{Float64}:
 3.37652e-316  5.16164e160
 0.0           3.37652e-316

In [3]:
m = CppEigenMatrix.Matrix{Float32}(2, 2)
m

2×2 Main.CppEigenMatrix.MatrixAllocated{Float32}:
 6.31802f-37  1.89395f34
 0.0          4.34382f-5

# Wrap ghostbasil's `BlockMatrix` class

+ We will be writing Julia bindings in `/home/users/bbchu/ghostbasil`
+ [this tutorial](https://www.youtube.com/watch?v=VoXmXtqLhdo&ab_channel=TheJuliaProgrammingLanguage) which wraps the Eigen library (starging at 1:13:00) was especially helpful.


### 1. Create `/home/users/bbchu/ghostbasil/julia/ghostbasil_wrap.cpp`

```cpp
#include "jlcxx/jlcxx.hpp"
#include "jlcxx/stl.hpp"
#include "ghostbasil/matrix/block_matrix.hpp"
#include "ghostbasil/matrix/block_group_ghost_matrix.hpp"

class OneBlockMatrixWrap
{
    jlcxx::ArrayRef<double, 2> orig_mat_; // original matrix
    Eigen::MatrixXd eigen_mat_; // convert to Eigen::MatrixXd
    ghostbasil::BlockMatrix<Eigen::Map<Eigen::MatrixXd>> block_mat_;

    static auto init_eigen_mat(jlcxx::ArrayRef<double, 2> mat, int_t rows, int_t cols) {
        Eigen::MatrixXd eigen_mat_(rows, cols);
        size_t k = 0;
        for (size_t i = 0; i < cols; i++) {
            for (size_t j = 0; j < rows; j++) {
                eigen_mat_(j, i) = mat[k++];
            }
        }
        return eigen_mat_;
    }

    static auto init_mat_list(Eigen::MatrixXd mat) {
        std::vector<Eigen::Map<Eigen::MatrixXd>> mat_list_;
        mat_list_.push_back(Eigen::Map<Eigen::MatrixXd>(mat.data(), 1, 1));
        return mat_list_;
    }

public: 
    OneBlockMatrixWrap(jlcxx::ArrayRef<double, 2> mat, int_t rows, int_t cols)
        : orig_mat_(mat),
        eigen_mat_(init_eigen_mat(mat, rows, cols)),
        block_mat_(init_mat_list(eigen_mat_))
    {
        std::cout << "row: " << rows << '\n';
        std::cout << "col: " << cols << '\n';
        std::cout << eigen_mat_ << std::endl;
    }
};

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.add_type<OneBlockMatrixWrap>("OneBlockMatrix")
        .constructor<jlcxx::ArrayRef<double, 2>, int_t, int_t>()
        ;
}

```

### 2: Create `/home/users/bbchu/ghostbasil/julia/CMakeLists.txt`:

```cmake
project(ghostbasil)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx REQUIRED)
find_package(Eigen3 3.3 REQUIRED NO_MODULE)
include_directories(${CMAKE_CURRENT_SOURCE_DIR}/../ghostbasil/include)

add_library(ghostbasil_wrap SHARED ghostbasil_wrap.cpp)
target_link_libraries(ghostbasil_wrap JlCxx::cxxwrap_julia Eigen3::Eigen)
install(TARGETS ghostbasil_wrap
  LIBRARY DESTINATION lib
  ARCHIVE DESTINATION lib
  RUNTIME DESTINATION lib)
```

### 3. Build the shared library (`.io`) file at `/home/users/bbchu/ghostbasil/julia/build`
```
mkdir build && cd build
cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override -DEigen3_DIR=/share/software/user/open/eigen/3.4.0/share/eigen3/cmake /home/users/bbchu/ghostbasil/julia
cmake --build . --config Release
```
### 4. Call from Julia

In [1]:
module ghostbasil
    using CxxWrap

    @wrapmodule("/home/users/bbchu/ghostbasil/julia/build/lib/libghostbasil_wrap")

    function __init__()
        @initcxx
    end
end

module CppEigenMatrix
    using CxxWrap

    @wrapmodule("/home/users/bbchu/ghostbasilwrap/build/lib/libeigen_matrix")

    function __init__()
        @initcxx
    end

    Base.size(m::Matrix) = (rows(m), cols(m))
    Base.IndexStyle(::Type{<:Matrix})=IndexCartesian()
end

Main.CppEigenMatrix

In [2]:
m1 = CppEigenMatrix.Matrix{Float64}(2, 2)

2×2 Main.CppEigenMatrix.MatrixAllocated{Float64}:
 3.28014e-316  5.16164e160
 0.0           3.28014e-316

In [3]:
m1 = CppEigenMatrix.Matrix{Float64}(2, 2)
m2 = CppEigenMatrix.Matrix{Float64}(3, 3)
m3 = CppEigenMatrix.Matrix{Float64}(4, 4)
blocks = [m1, m2, m3]

3-element Vector{Main.CppEigenMatrix.MatrixAllocated{Float64}}:
 [3.80498215e-316 5.483607035636684e247; 0.0 3.80498294e-316]
 [6.9128624369722e-310 5.9596773034521166e228 3.27782833e-316; 4.0467652e-316 1.76790384e-316 3.1870475e-316; 1.2774512412801808e238 1.63e-322 3.95e-322]
 [6.91286243697536e-310 1.7692483e-316 3.60302155e-316 0.0; 4.00469297e-316 2.1219957914e-313 4.2439915824e-314 0.0; 1.7689631e-316 2.97079410794e-313 1.7692483e-316 4.901812848809653e-24; 1.76896627e-316 3.81959242453e-313 8.2757835865e-313 3.985066447142008e-58]

In [4]:
p = 5
mat = rand(p, p)
Si_scaled = ghostbasil.OneBlockMatrix(mat, n, p)

LoadError: UndefVarError: n not defined

In [5]:
m = 5
Ci = rand(p, p)
Bi = BlockGroupGhostMatrix(Ci, Si_scaled, m+1)

LoadError: UndefVarError: BlockGroupGhostMatrix not defined

# Wrap ghostbasil's `ghostbasil` function

1. In our [code](https://github.com/biona001/GhostKnockoffGWAS/blob/main/src/ghostbasil_parallel.jl#L209C1-L211C69), we need to call `ghostbasil(A, r)` where `A` is a `BlockGroupGhostMatrix`.
2. The `BlockGroupGhostMatrix` class essentially requires a regular matrix and a `BlockMatrix`. We will pass these matrices directly into C++ and try to internally call the ghostbasil function from within C++. 

### 1. Create `/home/users/bbchu/ghostbasil/julia/ghostbasil_wrap.cpp`

```cpp
#include "jlcxx/jlcxx.hpp"
#include "jlcxx/stl.hpp"
#include "ghostbasil/optimization/basil.hpp"
#include "ghostbasil/matrix/block_matrix.hpp"
#include "ghostbasil/matrix/block_group_ghost_matrix.hpp"

static auto init_eigen_vec(jlcxx::ArrayRef<double, 1> vec) {
    Eigen::VectorXd eigen_vec(vec.size());
    for (size_t i = 0; i < vec.size(); i++) {
        eigen_vec(i) = vec[i];
    }
    return eigen_vec;
}

static auto init_eigen_mat(jlcxx::ArrayRef<double, 2> mat, int_t rows, int_t cols) {
    Eigen::MatrixXd eigen_mat_(rows, cols);
    size_t k = 0;
    for (size_t i = 0; i < cols; i++) {
        for (size_t j = 0; j < rows; j++) {
            eigen_mat_(j, i) = mat[k++];
        }
    }
    return eigen_mat_;
}

// should be the same as BlockMatrixWrap 
// https://github.com/biona001/ghostbasil/blob/master/R/inst/include/rcpp_block_matrix.hpp
// but it only wraps 1 (dense) matrix at a time. 
class OneBlockMatrixWrap
{
    jlcxx::ArrayRef<double, 2> orig_mat_;
    Eigen::MatrixXd eigen_mat_;
    std::vector<Eigen::MatrixXd> mat_list_;
    ghostbasil::BlockMatrix<Eigen::MatrixXd> block_mat_;
    const Eigen::Array<size_t, 2, 1> dim_;

    static auto init_mat_list(Eigen::MatrixXd mat, int_t rows, int_t cols) {
        std::vector<Eigen::MatrixXd> mat_list_;
        mat_list_.reserve(1); // wrapper is for only 1 block of matrix 
        mat_list_.emplace_back(mat);
        return mat_list_;
    }

public: 
    OneBlockMatrixWrap(jlcxx::ArrayRef<double, 2> mat, int_t rows, int_t cols)
        : orig_mat_(mat),
        eigen_mat_(init_eigen_mat(mat, rows, cols)),
        mat_list_(init_mat_list(eigen_mat_, rows, cols)),
        block_mat_(mat_list_),
        dim_(block_mat_.rows(), block_mat_.cols())
    {}

    // GHOSTBASIL_STRONG_INLINE
    const auto& internal() const { return block_mat_; }

    // For export only
    const Eigen::Array<size_t, 2, 1> dim_exp() const { return dim_; }
};

// adapted from https://github.com/JamesYang007/ghostbasil/blob/master/R/inst/include/rcpp_block_group_ghost_matrix.hpp#L7
class BlockGroupGhostMatrixWrap
{
public:
    using bmat_t = ghostbasil::BlockMatrix<Eigen::MatrixXd>;

private:
    jlcxx::ArrayRef<double, 2> orig_mat_S_;
    jlcxx::ArrayRef<double, 2> orig_mat_D_;
    Eigen::MatrixXd eigen_mat_S_;
    Eigen::MatrixXd eigen_mat_D_;
    const Eigen::MatrixXd orig_S_;
    const OneBlockMatrixWrap orig_D_;
    ghostbasil::BlockGroupGhostMatrix<Eigen::MatrixXd, bmat_t> gmat_;
    const Eigen::Array<size_t, 2, 1> dim_;

public:
    BlockGroupGhostMatrixWrap(
        jlcxx::ArrayRef<double, 2> S, // standard matrix from Julia
        jlcxx::ArrayRef<double, 2> D, // this will be converted to a BlockMatrix
        int_t m, // m is number of knockoffs
        int_t rows, // size(D, 1)
        int_t cols  // size(D, 2)
    )
        : orig_mat_S_(S),
          orig_mat_D_(D),
          eigen_mat_S_(init_eigen_mat(S, rows, cols)),
          eigen_mat_D_(init_eigen_mat(D, rows, cols)),
          orig_S_(eigen_mat_S_),
          orig_D_(orig_mat_D_, rows, cols),
          gmat_(orig_S_, 
                orig_D_.internal(), 
                m + 1), // in ghostbasil, the original variables counts as 1 knockoff
          dim_(gmat_.rows(), gmat_.cols())
    {}

    // GHOSTBASIL_STRONG_INLINE
    const auto& internal() const { return gmat_; }

    // For export only
    const Eigen::Array<size_t, 2, 1> dim_exp() const { return dim_; }
};

// Same function as `basil__` at 
// https://github.com/JamesYang007/ghostbasil/blob/master/R/src/rcpp_basil.cpp#L88
// Except: (a) this returns only the beta that corresponds to the last lambda value
// and (b) there is a lot less "checking" (so its behavior is unpredictable
// when user early terminates)
template <class AType>
void basil__(
        std::vector<double>& beta_i, // output result
        const AType& A, 
        const Eigen::Map<Eigen::VectorXd> r,
        double alpha,
        const Eigen::Map<Eigen::VectorXd> penalty,
        const Eigen::Map<Eigen::VectorXd> user_lmdas,
        size_t max_n_lambdas,
        size_t n_lambdas_iter,
        bool use_strong_rule,
        bool do_early_exit,
        size_t delta_strong_size,
        size_t max_strong_size,
        size_t max_n_cds,
        double thr,
        double min_ratio,
        size_t n_threads)
{
    using namespace ghostbasil::lasso;

    std::vector<Eigen::SparseVector<double>> betas;
    std::vector<double> lmdas;
    std::vector<double> rsqs;

    // slight optimization: reserve spaces ahead of time
    const size_t capacity = std::max(
        max_n_lambdas, static_cast<size_t>(user_lmdas.size())
    );
    betas.reserve(capacity);
    lmdas.reserve(capacity);
    rsqs.reserve(capacity);
    std::string error;

    try {
        basil(
                A, r, alpha, penalty, user_lmdas, max_n_lambdas, n_lambdas_iter,
                use_strong_rule, do_early_exit, delta_strong_size, max_strong_size, max_n_cds, thr, 
                min_ratio, n_threads,
                betas, lmdas, rsqs);
    }
    catch (const std::exception& e) {
        error = e.what();
    }

    // return the beta corresponding to the last lambda value
    Eigen::SparseVector<double> last_beta = betas.back();
    for (Eigen::SparseVector<double>::InnerIterator it(last_beta); it; ++it) {
        beta_i[it.index()] = it.value();
    }
}

// Same as `basil_block_group_ghost__` at
// https://github.com/JamesYang007/ghostbasil/blob/master/R/src/rcpp_basil.cpp#L291
// But we don't create a `BlockGroupGhostMatrix` in Julia. Instead, we
// pass `C` and `S` as Julia matrices into C++, convert `S` to a `BlockMatrix`
// with a single block, convert `C` to a Eigen matrix, and then pass `C` and `S`
// directly into `basil__`
void basil_block_group_ghost__(
        std::vector<double>& output_beta, // output result
        const jlcxx::ArrayRef<double, 2> C, // dense matrix
        const jlcxx::ArrayRef<double, 2> S, // becomes a BlockMatrix
        const jlcxx::ArrayRef<double, 1> r,
        const jlcxx::ArrayRef<double, 1> user_lmdas,
        int_t m, // number of knockoffs
        int_t p, // dimension of S and C (both square matrices)
        size_t max_n_lambdas,
        size_t n_lambdas_iter,
        bool use_strong_rule,
        bool do_early_exit,
        size_t delta_strong_size,
        size_t max_strong_size,
        size_t max_n_cds,
        double thr,
        double min_ratio,
        size_t n_threads)
{
    // convert C and S to instance of BlockGroupGhostMatrix
    auto gmw = BlockGroupGhostMatrixWrap(C, S, m, p, p);
    const auto& gm = gmw.internal();

    // default alpha = 1.0 and penalty = vector of 1s
    double alpha = 1.0;
    Eigen::VectorXd ones = Eigen::VectorXd::Ones((m+1) * p);
    Eigen::Map<Eigen::VectorXd> penalty(ones.data(), ones.size());

    // convert r and user_lmdas to Eigen::Map<Eigen::VectorXd>
    Eigen::VectorXd lmdas = init_eigen_vec(user_lmdas);
    Eigen::VectorXd r1 = init_eigen_vec(r);
    Eigen::Map<Eigen::VectorXd> lambdas(lmdas.data(), lmdas.size());
    Eigen::Map<Eigen::VectorXd> r2(r1.data(), r1.size());

    // call basil__ function defined above 
    basil__(
        output_beta, gm, r2, alpha, penalty, lambdas, max_n_lambdas,
        n_lambdas_iter, use_strong_rule, do_early_exit, delta_strong_size,
        max_strong_size, max_n_cds, thr, min_ratio, n_threads);
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.method("block_group_ghostbasil", [](
        const jlcxx::ArrayRef<double, 2> C, // dense matrix
        const jlcxx::ArrayRef<double, 2> S, // becomes a BlockMatrix
        const jlcxx::ArrayRef<double, 1> r,
        const jlcxx::ArrayRef<double, 1> user_lmdas,
        int_t m, // number of knockoffs
        int_t p, // dimension of S and C (both square matrices)
        size_t max_n_lambdas,
        size_t n_lambdas_iter,
        bool use_strong_rule,
        bool do_early_exit,
        size_t delta_strong_size,
        size_t max_strong_size,
        size_t max_n_cds,
        double thr,
        double min_ratio,
        size_t n_threads
    ){
        std::vector<double> output_beta((m+1) * p, 0.0);
        basil_block_group_ghost__(
            output_beta, C, S, r, user_lmdas, m, p, max_n_lambdas, n_lambdas_iter,
            use_strong_rule, do_early_exit, delta_strong_size, max_strong_size,
            max_n_cds, thr, min_ratio, n_threads);
        return output_beta;
    });
}
```

### 2: Create `/home/users/bbchu/ghostbasil/julia/CMakeLists.txt`:

```
project(ghostbasil)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx REQUIRED)
find_package(Eigen3 3.3 REQUIRED NO_MODULE)
include_directories(${CMAKE_CURRENT_SOURCE_DIR}/../ghostbasil/include)

add_library(ghostbasil_wrap SHARED ghostbasil_wrap.cpp)
target_link_libraries(ghostbasil_wrap JlCxx::cxxwrap_julia Eigen3::Eigen)
install(TARGETS ghostbasil_wrap
  LIBRARY DESTINATION lib
  ARCHIVE DESTINATION lib
  RUNTIME DESTINATION lib)
```

### 3. Build the shared library (`.io`) file at `/home/users/bbchu/ghostbasil/julia/build`

Note: `DCMAKE_INSTALL_PREFIX` points to the output of `CxxWrap.prefix_path()`

```
mkdir build && cd build
cmake \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_INSTALL_PREFIX=/home/groups/sabatti/.julia/artifacts/d25a86126e4d79572040fbf3f1fcda158a703a4a \
    -DEigen3_DIR=/share/software/user/open/eigen/3.4.0/share/eigen3/cmake \
    ../
cmake --build . --config Release
```

### 4. Call from Julia

Note: Julia version of the Jupyter notebook needs to be the same as the version that compiled the code (i.e. 1.8.4, otherwise we will get segfaults)

In [1]:
module Ghostbasil
using CxxWrap
using ghostbasil_jll

@wrapmodule(ghostbasil_jll.get_libghostbasil_wrap_path)
# @wrapmodule(() -> "/home/users/bbchu/ghostbasil/julia/build2/lib/libghostbasil_wrap")

function __init__()
    @initcxx
end

function block_group_ghostbasil(
    Ci::Matrix{Float64}, 
    Si_scaled::Matrix{Float64}, 
    r::Vector{Float64}, 
    lambda_path::Vector{Float64};
    m::Int = 5,
    max_n_lambdas::Int = 100,
    lambdas_iter::Int = 5,
    use_strong_rule::Bool = false,
    do_early_exit::Bool = true,
    delta_strong_size::Int = 500,
    max_strong_size::Int = (m+1)*size(Ci, 1),
    max_n_cds::Int = 100000,
    thr::Float64 = 1e-7,
    min_ratio::Float64 = 1e-2,
    n_threads::Int = 1,
    )
    # check for errors
    p = size(Ci, 1)
    size(Ci) == size(Si_scaled) || error("Expected size(Ci) == size(Si_scaled)")
    length(r) == (m+1)*p || error("Expected length(r) == $((m+1)*p)")
    issorted(lambda_path, rev=true) || 
        error("lambda_path should be sorted from largest to smallest")
    isapprox(Ci, Ci', atol=1e-8) || error("Ci is not symmetric")
    isapprox(Si_scaled, Si_scaled', atol=1e-8) || 
        error("Si_scaled is not symmetric")
    (m > 0) && (max_n_lambdas > 0) && (lambdas_iter > 0) && 
        (max_n_cds > 0) && (thr > 0) && (min_ratio > 0) || 
        error("Expected m, max_n_lambdas, lambdas_iter, max_n_cds, thr, min_ratio all to be > 0")

    # call C++ solver
    beta_i = block_group_ghostbasil(Ci, Si_scaled, r, lambda_path, 
        m, p, max_n_lambdas, lambdas_iter, use_strong_rule, do_early_exit, 
        delta_strong_size, max_strong_size, max_n_cds, thr, min_ratio, n_threads)

    # convert to Julia vector, free C++ memory, then return
    beta_i = copy.(beta_i)
    return beta_i
end

export block_group_ghostbasil

datadir(parts...) = joinpath(@__DIR__, "..", "data", parts...)

end

Main.Ghostbasil

In [2]:
# ml cmake/3.11.1 julia/1.8.4 eigen/3.4.0 R/4.0.2 gcc/7.1.0

using DelimitedFiles
using RCall

# a test region 
datadir = "/home/users/bbchu/ghostbasil/data"
Ci = readdlm(joinpath(datadir, "Ci.txt"))
Si_scaled = readdlm(joinpath(datadir, "Si_scaled.txt"))
r = readdlm(joinpath(datadir, "r.txt")) |> vec
lambda_path = readdlm(joinpath(datadir, "lambda_path.txt")) |> vec
Adiag = Ci + Si_scaled
Aoffdiag = Ci

# other arguments
m = 5
p = size(Ci, 1)
max_n_lambdas = 100
lambdas_iter = 5
use_strong_rule = false
do_early_exit = true
delta_strong_size = 500
max_strong_size = (m+1)*size(Ci, 1)
max_n_cds = 100000
thr = 1e-7
min_ratio = 1e-2
n_threads = 1

# julia wrapper of ghostbasil
# beta_i = ghostbasil.block_group_ghostbasil(Ci, Si_scaled, r, lambda_path, 
#     m, p, max_n_lambdas, lambdas_iter, use_strong_rule, do_early_exit, 
#     delta_strong_size, max_strong_size, max_n_cds, thr, min_ratio, n_threads)
@time beta_i = Ghostbasil.block_group_ghostbasil(Ci, Si_scaled, r, lambda_path)

# import data in C++ before running basil__ (eigen matrix, BlockMatrix, and BlockGroupGhostMatrix)
C2 = readdlm(joinpath(datadir, "C2.txt"), Float64)
S2 = readdlm(joinpath(datadir, "S2.txt"), Float64)
A2 = readdlm(joinpath(datadir, "A2.txt"), Float64)
Adiag2 = A2[1:p, 1:p]
Aoffdiag2 = A2[p+1:2p, 1:p]
@show all(isapprox.(C2, Ci, atol=5e-7))
@show all(isapprox.(S2, Si_scaled, atol=5e-7))
@show all(isapprox.(Adiag, Adiag2, atol=5e-7))
@show all(isapprox.(Aoffdiag, Aoffdiag2, atol=5e-7))

# check answer
ncores = 1
A_scaling_factor = 0.01
@rput Ci Si_scaled r m ncores A_scaling_factor lambda_path
R"""
library(ghostbasil)

# make S into a block matrix with a single block
Si_scaled <- BlockMatrix(list(Si_scaled))
# form Bi
Bi <- BlockGroupGhostMatrix(Ci, Si_scaled, m+1)
# run ghostbasil on one specific lambda
result <- ghostbasil(Bi, r, delta.strong.size=500, 
    max.strong.size = nrow(Bi), n.threads=ncores, 
    user.lambdas=lambda_path, use.strong.rule=F)
beta_i_true <- result$betas[,ncol(result$betas)]
0.0 # prevents printing beta_i_true
"""
@rget beta_i_true;
@show count(!iszero, beta_i_true);
@show count(!iszero, beta_i);
@show all(beta_i_true .≈ beta_i);

  0.035640 seconds (335 allocations: 5.518 MiB, 27.52% compilation time)
all(isapprox.(C2, Ci, atol = 5.0e-7)) = true
all(isapprox.(S2, Si_scaled, atol = 5.0e-7)) = true
all(isapprox.(Adiag, Adiag2, atol = 5.0e-7)) = true
all(isapprox.(Aoffdiag, Aoffdiag2, atol = 5.0e-7)) = true


┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /home/groups/sabatti/.julia/packages/RCall/YrsKg/src/io.jl:172


count(!iszero, beta_i_true) = 14
count(!iszero, beta_i) = 14
all(beta_i_true .≈ beta_i) = true


Another test: this "large" region gives segfaults

In [3]:
# ml cmake/3.11.1 julia/1.8.4 eigen/3.4.0 R/4.0.2 gcc/7.1.0

using DelimitedFiles
using RCall

# region where I keep getting segfaults
testdata_dir = "/home/groups/sabatti/.julia/dev/Ghostbasil/data"
Ci = readdlm(joinpath(testdata_dir, "C2.txt"))
Si_scaled = readdlm(joinpath(testdata_dir, "S2.txt"))
r = readdlm(joinpath(testdata_dir, "r2.txt")) |> vec
lambda_path = readdlm(joinpath(testdata_dir, "lambda_path2.txt")) |> vec

# ghostbasil
@time beta_i = Ghostbasil.block_group_ghostbasil(Ci, Si_scaled, r, lambda_path)

# check answer
ncores = 1
A_scaling_factor = 0.01
@rput Ci Si_scaled r m ncores A_scaling_factor lambda_path
R"""
library(ghostbasil)

# make S into a block matrix with a single block
Si_scaled <- BlockMatrix(list(Si_scaled))
# form Bi
Bi <- BlockGroupGhostMatrix(Ci, Si_scaled, m+1)
# run ghostbasil on one specific lambda
result <- ghostbasil(Bi, r, delta.strong.size=500, 
    max.strong.size = nrow(Bi), n.threads=ncores, 
    user.lambdas=lambda_path, use.strong.rule=F)
beta_i_true <- result$betas[,ncol(result$betas)]
0.0 # prevents printing beta_i_true
"""
@rget beta_i_true;
@show count(!iszero, beta_i_true);
@show count(!iszero, beta_i);
@show all(beta_i_true .≈ beta_i);

  0.515050 seconds (7 allocations: 60.579 MiB)
count(!iszero, beta_i_true) = 15
count(!iszero, beta_i) = 15
all(beta_i_true .≈ beta_i) = true


Final test: completely random simulated data

In [4]:
p = 10
x = randn(p, p)
y = randn(p, p)
Ci = x'*x
Si_scaled = y'*y
r = randn(6p)
lambda_path = [0.1, 0.05, 0.01, 0.001]

# ghostbasil julia
beta_i = Ghostbasil.block_group_ghostbasil(Ci, Si_scaled, r, lambda_path)

@show all(length(beta_i) == 6p)


# check answer
ncores = 1
@rput Ci Si_scaled r m ncores lambda_path
R"""
library(ghostbasil)

# make S into a block matrix with a single block
Si_scaled <- BlockMatrix(list(Si_scaled))
# form Bi
Bi <- BlockGroupGhostMatrix(Ci, Si_scaled, m+1)
# run ghostbasil on one specific lambda
result <- ghostbasil(Bi, r, delta.strong.size=500, 
    max.strong.size = nrow(Bi), n.threads=ncores, 
    user.lambdas=lambda_path, use.strong.rule=F)
beta_i_true <- result$betas[,ncol(result$betas)]
0.0 # prevents printing beta_i_true
"""
@rget beta_i_true;
@show count(!iszero, beta_i_true);
@show count(!iszero, beta_i);
@show all(beta_i_true .≈ beta_i);

all(length(beta_i) == 6p) = true
count(!iszero, beta_i_true) = 60
count(!iszero, beta_i) = 60
all(beta_i_true .≈ beta_i) = true


# Make jll package via BinaryBuilder

```julia
using BinaryBuilder
state = BinaryBuilder.run_wizard()
```

1. Must run this on mac (rather than on Sherlock/Hoffman2 cluster), see [this issue](https://github.com/JuliaPackaging/BinaryBuilder.jl/issues/1306)
2. We only build for linux and Mac, since ghostbasil doesn't currently build on windows
3. Source code
    + URL: https://github.com/biona001/ghostbasil
    + commit: use latest
4. When asked whether there are additional "binary dependencies", my understanding is that we at least need Julia itself and the libcxxwrap library to enable interfacing with C++. Since ghosbasil also requires `Eigen` package, we actually have 3 binary dependencies:
    + libcxxwrap_julia_jll
    + Eigen_jll
    + libjulia_jll

Note `libcxxwrap_julia_jll` should have a compat entry in the `build_tarballs.jl`, i.e. `Dependency("libcxxwrap_julia_jll"; compat = "0.11.2")`. An incompatible version might lead to extremely hard to debug segfaults. 

5. When asked whether to customize the set of compilers, we used
    + GCC: 7.1.0 (note: according to [BinaryBuilder docs](https://github.com/JuliaPackaging/Yggdrasil/blob/0d38df8bc8ad10cff5fba1c19a5932a84286fcd2/CONTRIBUTING.md#compatibility-tips), we should use oldest possible gcc)
    + LLVM: 16.0.6
6. build script:

```
mkdir ghostbasil/julia/build
cd ghostbasil/julia/build
cmake -DJulia_PREFIX=$prefix -DCMAKE_INSTALL_PREFIX=$prefix -DCMAKE_FIND_ROOT_PATH=$prefix -DCMAKE_TOOLCHAIN_FILE=${CMAKE_TARGET_TOOLCHAIN} -DEigen3_DIR=$prefix/share/eigen3/cmake -DCMAKE_BUILD_TYPE=Release ..
make
make install
install_license $WORKSPACE/srcdir/ghostbasil/R/LICENSE.md
```

7. Before opening a PR, lets first test the `build_tarballs.jl` by seeing if we can [build jll package onto github](https://docs.binarybuilder.org/stable/FAQ/#Can-I-publish-a-JLL-package-locally-without-going-through-Yggdrasil?) (Note this takes 1-2h):
```
julia build_tarballs.jl --debug --verbose --deploy="biona001/ghostbasil_jll.jl"
```

8. Step 7 should have generated a new release at `github.com/biona001/ghostbasil_jll.jl` package. Try using it via
```
using Pkg
Pkg.add(url="https://github.com/biona001/ghostbasil_jll.jl")
Pkg.add(url="https://github.com/biona001/Ghostbasil.jl")
Pkg.add(url="https://github.com/biona001/GhostKnockoffGWAS")
```

The final `build_tarballs.jl` looks like (note this script is typically stored under `.julia/dev/Yggdrasil`)
```julia
# Note that this script can accept some limited command-line arguments, run
# `julia build_tarballs.jl --help` to see a usage message.
using BinaryBuilder, Pkg

name = "ghostbasil"
version = v"0.0.14"

# Collection of sources required to complete build
sources = [
    GitSource("https://github.com/biona001/ghostbasil.git", "50aef6a0f7810b2a8a41e6ee36079950fcf54313")
]

# Bash recipe for building across all platforms
script = raw"""
cd $WORKSPACE/srcdir
mkdir -p ghostbasil/julia/build
cd ghostbasil/julia/build

cmake \
    -DJulia_PREFIX=$prefix \
    -DCMAKE_INSTALL_PREFIX=$prefix \
    -DCMAKE_FIND_ROOT_PATH=$prefix \
    -DCMAKE_TOOLCHAIN_FILE=${CMAKE_TARGET_TOOLCHAIN} \
    -DEigen3_DIR=$prefix/share/eigen3/cmake \
    -DCMAKE_BUILD_TYPE=Release \
    ../

make
make install

# install license
install_license $WORKSPACE/srcdir/ghostbasil/R/LICENSE.md
"""

# These are the platforms we will build for by default, unless further
# platforms are passed in on the command line
julia_versions = [
    v"1.8.0", v"1.8.1", v"1.8.2", v"1.8.3", v"1.8.4", v"1.8.5",
    v"1.9.0", v"1.9.1", v"1.9.2", v"1.9.3", v"1.9.4", 
    v"1.10.0"
]
working_platforms = [ # ghostbasil won't work on windows, and currently fails on mac
    Platform("x86_64", "linux"; libc = "glibc"),
    Platform("aarch64", "linux"; libc = "glibc"),
    Platform("powerpc64le", "linux"; libc = "glibc"),
    Platform("x86_64", "linux"; libc = "musl"),
    Platform("aarch64", "linux"; libc = "musl"),
]

# expand platforms to specify julia version explicitly
# see https://github.com/JuliaInterop/CxxWrap.jl/issues/395
platforms = Platform[]
for p in working_platforms, julia_version in julia_versions
    p["julia_version"] = string(julia_version)
    push!(platforms, deepcopy(p))
end
platforms = expand_cxxstring_abis(platforms)

# The products that we will ensure are always built
products = [
    LibraryProduct("libghostbasil_wrap", :libghostbasil_wrap)
]

# Dependencies that must be installed before this package can be built
dependencies = [
    Dependency("libcxxwrap_julia_jll"; compat = "0.11.2"),
    BuildDependency("Eigen_jll"),
    BuildDependency("libjulia_jll")
]

# Build the tarballs, and possibly a `build.jl` as well.
build_tarballs(ARGS, name, version, sources, script, platforms, products, 
    dependencies; julia_compat="1.9", preferred_gcc_version = v"7.1.0")

```

# Write `ghostbasil.jl` package 

In [1]:
# ml cmake/3.11.1 gcc/12.1.0 julia/1.8.4 eigen/3.4.0 R/4.0.2

using ghostbasil
using DelimitedFiles
using RCall

# read first 4 args for ghostbasil
datadir = "/home/users/bbchu/ghostbasil/data"
Ci = readdlm(joinpath(datadir, "Ci.txt"))
Si_scaled = readdlm(joinpath(datadir, "Si_scaled.txt"))
r = readdlm(joinpath(datadir, "r.txt")) |> vec
lambda_path = readdlm(joinpath(datadir, "lambda_path.txt")) |> vec
Adiag = Ci + Si_scaled
Aoffdiag = Ci

# julia wrapper of ghostbasil
beta_i = block_group_ghostbasil(Ci, Si_scaled, r, lambda_path)

# check answer
ncores = 1
A_scaling_factor = 0.01
m = 5
@rput Ci Si_scaled r m ncores A_scaling_factor lambda_path
R"""
library(ghostbasil)

# make S into a block matrix with a single block
Si_scaled <- BlockMatrix(list(Si_scaled))
# form Bi
Bi <- BlockGroupGhostMatrix(Ci, Si_scaled, m+1)
# run ghostbasil on one specific lambda
result <- ghostbasil(Bi, r, delta.strong.size=500, 
    max.strong.size = nrow(Bi), n.threads=ncores, 
    user.lambdas=lambda_path, use.strong.rule=F)
beta_i_true <- result$betas[,ncol(result$betas)]
0.0 # prevents printing beta_i_true
"""
@rget beta_i_true;
@show count(!iszero, beta_i_true);
@show count(!iszero, beta_i);
@show all(beta_i_true .≈ beta_i);

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /home/groups/sabatti/.julia/packages/RCall/gOwEW/src/io.jl:172


count(!iszero, beta_i_true) = 14
count(!iszero, beta_i) = 14
all(beta_i_true .≈ beta_i) = true


In [5]:
beta_i[findall(!iszero, beta_i)] |> println

[-0.0005451576807477974, -0.0002947665358369119, -0.014810108231841996, -0.00043238505905553924, -0.006856782960094195, 0.0016850133196618926, -0.00019274743491070077, -0.0003841629682186662, -0.0005758897376517483, -0.0016095272344540995, 0.0020146776266541312, -0.001666736964454875, 0.0004269118551173737, 0.00010746291189108347]


In [7]:
beta_i_true[findall(!iszero, beta_i_true)] |> println

[-0.0005451576807477974, -0.0002947665358369119, -0.014810108231841996, -0.00043238505905553924, -0.006856782960094195, 0.0016850133196618926, -0.00019274743491070077, -0.0003841629682186662, -0.0005758897376517483, -0.0016095272344540995, 0.0020146776266541312, -0.001666736964454875, 0.0004269118551173737, 0.00010746291189108347]


## PackageCompiler.jl